In [48]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.externals import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
from sklearn.naive_bayes import MultinomialNB

corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
posts = pd.read_csv('posts_count_me_pol.csv.gz', compression='gzip', error_bad_lines=False)
stopwords = stopwords.words("portuguese")
feature_names = pd.read_csv('feature_names.csv')

In [61]:
corpus = pd.read_csv('corpus.csv.gz', compression='gzip')
#corpus = corpus[corpus['_golden'] == False]
corpus = corpus[corpus['_trusted_judgments'] == 3]
corpus = corpus.reset_index()
corpus.shape

(916, 28)

In [62]:
# fix labels to binary
lb = preprocessing.LabelBinarizer(neg_label=1, pos_label=2)
target = lb.fit_transform(corpus['qual_a_melhor_classificao_para_esse_texto'].values)
c, r = target.shape
target = target.reshape(c,)
corpus['class'] = target

In [51]:
corpus[['qual_a_melhor_classificao_para_esse_texto','class']].sample(5)

,qual_a_melhor_classificao_para_esse_texto,class
460,outro,2
556,diario,1
724,diario,1
661,diario,1
718,diario,1


In [63]:
posts['class'] = 0
column_idx = posts.columns.get_loc('class')

for idx_corpus in corpus.index:
    postid = corpus.iloc[idx_corpus].postid
    idx_post = posts[posts['postID']==postid].index
    posts.iloc[idx_post,column_idx] = corpus.iloc[idx_corpus]['class']

In [64]:
vectorizer = TfidfVectorizer(ngram_range=(1,1), strip_accents='unicode', 
                             stop_words=stopwords, vocabulary=feature_names['0'].values)
data = vectorizer.fit_transform(posts.content)
data.shape

(178060, 800)

In [65]:
matrixPD = pd.DataFrame(data.toarray())
matrixPD['class'] = posts['class']
matrixPD = matrixPD[matrixPD['class'] > 0]
matrixPD.shape

label = matrixPD['class'].values
features = matrixPD.drop('class',axis=1)

In [66]:
from sklearn.model_selection import cross_val_score
model = MultinomialNB()
model.fit(features, label)
cross_val_score(model, features, label, cv=10, scoring='accuracy').mean()

0.70183574879227062

In [67]:
posts['prediction'] = model.predict(data)

In [68]:
len(posts[posts['prediction']==1])

134360